# Machine Learning Engineer Nanodegree
## Capstone Project - Data Preparation
Purpose of this notebook is to access all data sources, gather, clean and prepare all data to be used in capstone project. We will focus on **DMND students** that enrolled in any moment after **2017-04-01** and before **2017-08-10**. We will use as total universe users who are registered and visited DMND NDOP at least once in this period.

## Getting the universe

In [60]:
import pandas as pd
import numpy as np
import udb
import datetime

start_date = datetime.date(year=2017, month=4, day=1)
end_date = datetime.date(year=2017, month=8, day=10)

br = udb.get_ebdb_engine()
us = udb.get_analytics_engine()

all_accounts_who_visited = pd.read_sql_query("""
SELECT DISTINCT fbi.email
FROM frontend_brazil.pages fbp
  LEFT JOIN frontend_brazil.identifies fbi ON fbp.anonymous_id = fbi.anonymous_id
WHERE fbp.received_at >= '{}' AND fbp.received_at <= '{}'
""".format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')), con=us)

all_accounts_who_visited.shape[0]

32544

Now, let's get who became **paying student** in this period - the target feature we will train our model to predict.

## Paying students

In [61]:
paying_students = pd.read_sql_query("""
SELECT
  au.email
FROM payment_app_subscription ps
  INNER JOIN payment_app_product pp ON ps.product_id = pp.id
  INNER JOIN auth_user au ON au.id = ps.user_id
WHERE (status = 'active' OR status = 'payment_credit_retry')
      AND pp.code like 'nd018%%'
      AND register_date >= '{}'
      AND register_date <= '{}'
""".format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')), con=br)

paying_students.shape[0]

3599

Next step is to merge both into one dataframe, with **email** and target feature **is_paying_student**, showing 0 or 1.

In [62]:
paying_students['is_paying_student'] = 1
x = pd.merge(all_accounts_who_visited, paying_students, how='left', on='email')

In [63]:
x['is_paying_student'] = x['is_paying_student'].fillna(0).astype(int)
print(x['is_paying_student'].sum(), x.shape[0])

2951 32544


As we can see above, despite the fact we registered **3,601** paying students in Brazil database, there's data about NDOP visits of only **2,952** of them. We will move forward with these **2,952**.
From now on, we will start adding features we think can explain their behaviour in enrolling.

## How old are these accounts?
Our first hypothesis is that the likelihood of a student to enroll is influenced by how recent they have an account registered at Udacity.

In [64]:
x['email'] = x['email'].astype(str)
email_query = '('
for index, row in x.iterrows():
    if index == x['email'].shape[0] - 1:
        email_query = email_query + "'" + row['email'] + "')"
    else:
        email_query = email_query + "'" + row['email'] + "', "

In [65]:
emails_dates_joined = pd.read_sql_query("SELECT email, date_joined FROM auth_user WHERE email in {}".format(email_query), con=br)
emails_dates_joined.shape[0]

31991

As you can see above, we were not able to find all join dates from the **32,543** accounts in our universe, we found only **31,990**. Let's see if we have a better luck looking into US database:

In [66]:
emails_dates_joined_us = pd.read_sql_query("SELECT email, created_at FROM analytics_tables.accounts WHERE email in {}".format(email_query), con=us)
emails_dates_joined_us.shape[0]

31845

Strangely enough, we found even less data in US database. Let's move forward with Brazil data and disregard rows without join date:

In [67]:
result = pd.merge(x, emails_dates_joined, how='inner', on='email')
result.shape[0]

31991

This is not quite what we want: let's get the age in days, assuming today is the last day of the period:

In [68]:
def calculate_age(row):
    d2 = row['date_joined'].to_pydatetime().date()
    d1 = end_date
    return abs((d2 - d1).days)
    
result['age_in_days'] = result.apply(calculate_age, axis=1)
result.shape[0]

31991

In [69]:
x = pd.merge(x, result[['email', 'age_in_days']], how='inner', on='email')
x.shape[0]

31991

In [70]:
x.head()

,email,is_paying_student,age_in_days
0,zagg@zagg.com.br,0,407
1,matheus.sfreire@gmail.com,0,406
2,raulg.sales@hotmail.com,0,401
3,prof.sergio.costa@gmail.com,0,403
4,sergioaguiar@yahoo.com,0,375


## How many times has each user enrolled in a webinar?
Doing webinars is one of our key strategies to engage leads. Let's check if it really leads to conversion.

In [71]:
webinar_enrollments = pd.read_sql_query("""
SELECT email, COUNT(id)
FROM brazil_events.event_sign_up
WHERE enrollment_date >= '{}' and enrollment_date <= '{}'
GROUP BY email
""".format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')), con=us)

webinar_enrollments.shape[0]

23858

In [72]:
x = pd.merge(x, webinar_enrollments, how='left', on='email')
x.shape[0]

31991

In [73]:
x['webinar_enrollments'] = x['count'].fillna(0).astype(int)
x = x[['email', 'is_paying_student', 'age_in_days', 'webinar_enrollments']]
x.head()

,email,is_paying_student,age_in_days,webinar_enrollments
0,zagg@zagg.com.br,0,407,0
1,matheus.sfreire@gmail.com,0,406,0
2,raulg.sales@hotmail.com,0,401,0
3,prof.sergio.costa@gmail.com,0,403,0
4,sergioaguiar@yahoo.com,0,375,0


## How many times has each user enrolled in a free course?
There are people who believe free courses leads to paying students. Let's see if that's true.

In [74]:
free_course_enrollments = pd.read_sql_query("""
SELECT
  ac.email,
  count(ce.course_key) as course_enrollments
FROM analytics_tables.course_enrollments ce 
INNER JOIN analytics_tables.accounts ac on ce.user_id = ac.user_id
  WHERE ce.join_time <= '{}' AND ac.email IN {}
GROUP BY ac.email
""".format(end_date.strftime('%Y-%m-%d'), email_query), con=us)

free_course_enrollments.shape[0]

14831

In [75]:
x = pd.merge(x, free_course_enrollments, how='left', on='email')
x.shape[0]

31991

In [76]:
x['course_enrollments'] = x['course_enrollments'].fillna(0).astype(int)
x.head()

,email,is_paying_student,age_in_days,webinar_enrollments,course_enrollments
0,zagg@zagg.com.br,0,407,0,1
1,matheus.sfreire@gmail.com,0,406,0,3
2,raulg.sales@hotmail.com,0,401,0,4
3,prof.sergio.costa@gmail.com,0,403,0,68
4,sergioaguiar@yahoo.com,0,375,0,3


## How many times has each user navigated to each of our key pages?
The navigation pattern should be considered in predicting whether a user will become a paying student or not.

In [77]:
all_visits = pd.read_sql_query("""
SELECT
  DISTINCT fbp.id,
  fbp.path,
  fbp.referrer,
  fbp.context_user_agent,
  fbi.email
FROM frontend_brazil.pages fbp
  LEFT JOIN frontend_brazil.identifies fbi ON fbi.anonymous_id = fbp.anonymous_id
WHERE fbp.received_at <= '{}' AND fbi.email IN {}
""".format(end_date.strftime('%Y-%m-%d'), email_query), con=us)

all_visits.shape[0]

1591460

In [78]:
all_visits.head()

,id,path,referrer,context_user_agent,email
0,ajs-aa70c5fefa25862449ee7920762c2e07,/,https://classroom.udacity.com/courses/st101/le...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,israellj7@hotmail.com
1,ajs-5bf0486312579d6d4f317a6dfd00c079,/account/billing/,https://br.udacity.com/account/,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,mateus.cassaniga@gmail.com
2,ajs-ea7e814d419cd155634d4fc44107b040,/course/react-nanodegree--nd019/,https://br.udacity.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,daviresio@gmail.com
3,ajs-b59399b7407547e39af90530b74951ac,/course/deep-learning-nanodegree-foundation--n...,https://br.udacity.com/nanodegree/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,andre.gallacci@gmail.com
4,ajs-d81c5112b08ad89a075b31788fb785be,/course/intro-to-data-science--ud359/,https://www.google.com.br/,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,ALINNE.MARIE@OUTLOOK.COM


In [79]:
visits = all_visits.copy()

In [80]:
visits['is_home'] = (visits['path'] == '/').astype(int)
visits['is_ndop'] = (visits['path'].str.contains('--nd')).astype(int)
visits['is_catalog_all'] = (visits['path'] == '/courses/all/').astype(int)
visits['is_catalog_nanodegrees'] = (visits['path'] == '/courses/nanodegrees/').astype(int)
visits['is_nanodegree_home'] = (visits['path'] == '/nanodegree/').astype(int)
visits['is_fcop_ud'] = (visits['path'].str.contains('--ud')).astype(int)
visits['is_fcop_cs'] = (visits['path'].str.contains('--cs')).astype(int)
visits['is_fcop_st'] = (visits['path'].str.contains('--st')).astype(int)
visits['is_signin'] = (visits['path'].str.contains('/signin/')).astype(int)
visits['is_event'] = (visits['path'].str.contains('/events/')).astype(int)
visits['is_50back'] = (visits['path'] == '/nanodegree/50-back/').astype(int)
visits['is_tech_requirements'] = (visits['path'] == '/tech-requirements//').astype(int)
visits['is_contact'] = (visits['path'] == '/contact/').astype(int)
visits['is_us'] = (visits['path'] == '/us/').astype(int)
visits['is_jobs'] = (visits['path'] == '/jobs/').astype(int)
visits['is_legal'] = (visits['path'] == '/legal/').astype(int)
visits['is_hire_talent'] = (visits['path'] == '/hire-talent/').astype(int)
visits['is_business'] = (visits['path'] == '/business/').astype(int)
visits['is_success'] = (visits['path'] == '/success/').astype(int)
visits['is_payment'] = (visits['path'] == '/payment/').astype(int)
visits['is_android'] = (visits['path'].str.contains('/android/')).astype(int)
visits['is_ai'] = (visits['path'].str.contains('/ai/')).astype(int)
visits['is_drive'] = (visits['path'].str.contains('/drive/')).astype(int)
visits['is_robotics'] = (visits['path'].str.contains('/robotics/')).astype(int)
visits['is_checkout'] = (visits['path'].str.contains('/checkout')).astype(int)

In [81]:
visits.head().transpose()

,0,1,2,3,4
id,ajs-aa70c5fefa25862449ee7920762c2e07,ajs-5bf0486312579d6d4f317a6dfd00c079,ajs-ea7e814d419cd155634d4fc44107b040,ajs-b59399b7407547e39af90530b74951ac,ajs-d81c5112b08ad89a075b31788fb785be
path,/,/account/billing/,/course/react-nanodegree--nd019/,/course/deep-learning-nanodegree-foundation--n...,/course/intro-to-data-science--ud359/
referrer,https://classroom.udacity.com/courses/st101/le...,https://br.udacity.com/account/,https://br.udacity.com/,https://br.udacity.com/nanodegree/,https://www.google.com.br/
context_user_agent,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
email,israellj7@hotmail.com,mateus.cassaniga@gmail.com,daviresio@gmail.com,andre.gallacci@gmail.com,ALINNE.MARIE@OUTLOOK.COM
is_home,1,0,0,0,0
is_ndop,0,0,1,1,0
is_catalog_all,0,0,0,0,0
is_catalog_nanodegrees,0,0,0,0,0
is_nanodegree_home,0,0,0,0,0


In [82]:
def fix_is_ndop(row):
    if row['is_checkout'] == 1:
        return 0
    else:
        return row['is_ndop']

visits['is_ndop'] = visits.apply(fix_is_ndop, axis=1)

In [83]:
import re

def is_mobile(row):
    if re.search('/Android|webOS|iPhone|iPad|iPod|BlackBerry|IEMobile|Opera Mini/', str(row['context_user_agent'])):
        return 1
    else:
        return 0

visits['is_mobile'] = visits.apply(is_mobile, axis=1)

In [84]:
visits['referrer'] = visits['referrer'].fillna('')
visits['is_referrer_google'] = (visits['referrer'].str.contains('.google.')).astype(int)
visits['is_referrer_facebook'] = (visits['referrer'].str.contains('.facebook.')).astype(int)
visits['is_referrer_live'] = (visits['referrer'].str.contains('.live.')).astype(int)
visits['is_referrer_infomoney'] = (visits['referrer'].str.contains('.infomoney.')).astype(int)
visits['is_referrer_catracalivre'] = (visits['referrer'].str.contains('.catracalivre.')).astype(int)
visits['is_referrer_android'] = (visits['referrer'].str.contains('.android.')).astype(int)
visits['is_referrer_anhanguera'] = (visits['referrer'].str.contains('anhanguera.')).astype(int)
visits['is_referrer_linkedin'] = (visits['referrer'].str.contains('.linkedin.')).astype(int)
visits['is_referrer_instagram'] = (visits['referrer'].str.contains('.instagram.')).astype(int)
visits['is_referrer_cbsi'] = (visits['referrer'].str.contains('.cbsi.')).astype(int)
visits['is_referrer_tecmundo'] = (visits['referrer'].str.contains('.tecmundo.')).astype(int)
visits['is_referrer_bing'] = (visits['referrer'].str.contains('.bing.')).astype(int)
visits['is_referrer_computerworld'] = (visits['referrer'].str.contains('.computerworld.')).astype(int)
visits['is_referrer_github'] = (visits['referrer'].str.contains('.github.')).astype(int)

In [85]:
visits.drop('path', axis=1, inplace=True)
visits.drop('referrer', axis=1, inplace=True)
visits.drop('context_user_agent', axis=1, inplace=True)
visits.head().transpose()

,0,1,2,3,4
id,ajs-aa70c5fefa25862449ee7920762c2e07,ajs-5bf0486312579d6d4f317a6dfd00c079,ajs-ea7e814d419cd155634d4fc44107b040,ajs-b59399b7407547e39af90530b74951ac,ajs-d81c5112b08ad89a075b31788fb785be
email,israellj7@hotmail.com,mateus.cassaniga@gmail.com,daviresio@gmail.com,andre.gallacci@gmail.com,ALINNE.MARIE@OUTLOOK.COM
is_home,1,0,0,0,0
is_ndop,0,0,1,1,0
is_catalog_all,0,0,0,0,0
is_catalog_nanodegrees,0,0,0,0,0
is_nanodegree_home,0,0,0,0,0
is_fcop_ud,0,0,0,0,1
is_fcop_cs,0,0,0,0,0
is_fcop_st,0,0,0,0,0


Now that we have a dataframe with all visits from our universe of users transformed in features, let's aggregate by email:

In [86]:
f = {
    'id': ['count'],
    'is_home': ['sum'],
    'is_ndop': ['sum'],
    'is_catalog_all': ['sum'],
    'is_catalog_nanodegrees': ['sum'],
    'is_nanodegree_home': ['sum'],
    'is_fcop_ud': ['sum'],
    'is_fcop_cs': ['sum'],
    'is_fcop_st': ['sum'],
    'is_signin': ['sum'],
    'is_event': ['sum'],
    'is_50back': ['sum'],
    'is_tech_requirements': ['sum'],
    'is_contact': ['sum'],
    'is_us': ['sum'],
    'is_jobs': ['sum'],
    'is_legal': ['sum'],
    'is_hire_talent': ['sum'],
    'is_business': ['sum'],
    'is_success': ['sum'],
    'is_payment': ['sum'],
    'is_android': ['sum'],
    'is_ai': ['sum'],
    'is_drive': ['sum'],
    'is_robotics': ['sum'],
    'is_checkout': ['sum'],
    'is_mobile': ['sum'],
    'is_referrer_google': ['sum'],
    'is_referrer_facebook': ['sum'],
    'is_referrer_live': ['sum'],
    'is_referrer_infomoney': ['sum'],
    'is_referrer_catracalivre': ['sum'],
    'is_referrer_android': ['sum'],
    'is_referrer_anhanguera': ['sum'],
    'is_referrer_linkedin': ['sum'],
    'is_referrer_instagram': ['sum'],
    'is_referrer_cbsi': ['sum'],
    'is_referrer_tecmundo': ['sum'],
    'is_referrer_bing': ['sum'],
    'is_referrer_computerworld': ['sum'],
    'is_referrer_github': ['sum']
}
grouped = visits.groupby('email', as_index=False).agg(f)

In [87]:
grouped.columns = grouped.columns.droplevel(-1)

In [88]:
grouped.rename(columns={'id': 'count_visits'}, inplace=True)
grouped.head().transpose()

,0,1,2,3,4
email,+557588971838@gmail.com,00hf11@gmail.com,01bertoferreira@gmail.com,02001gp@gmail.com,07019312.das@gmail.com
is_checkout,0,2,1,0,0
is_referrer_linkedin,0,0,0,0,0
is_ai,0,0,0,0,0
is_fcop_ud,0,5,0,0,0
is_fcop_cs,0,1,0,0,0
is_referrer_github,0,0,0,0,0
is_referrer_facebook,0,0,0,0,0
is_legal,0,0,0,0,0
is_mobile,0,0,0,0,0


Now, let's put everything together:

In [89]:
result = pd.merge(x, grouped, how='inner', on='email')
result.shape

(31991, 46)

In [90]:
result.head().transpose()

,0,1,2,3,4
email,zagg@zagg.com.br,matheus.sfreire@gmail.com,raulg.sales@hotmail.com,prof.sergio.costa@gmail.com,sergioaguiar@yahoo.com
is_paying_student,0,0,0,0,0
age_in_days,407,406,401,403,375
webinar_enrollments,0,0,0,0,0
course_enrollments,1,3,4,68,3
is_checkout,0,2,0,0,2
is_referrer_linkedin,0,0,0,0,0
is_ai,0,0,0,0,0
is_fcop_ud,0,24,20,27,5
is_fcop_cs,0,1,10,6,0


## How many emails each lead opened?
One of the key strategies we use to nurture leads is email marketing. Let's see if it is working.

In [91]:
email_data = pd.read_sql_query("""
SELECT context_traits_email AS email, id
FROM vero_email.email_opened 
WHERE received_at <= '{}'
AND context_traits_email IN {}
""".format(end_date.strftime('%Y-%m-%d'), email_query), con=us)

email_data.shape[0]

941258

In [92]:
grouped_email_data = email_data.groupby('email')['id'].count()
grouped_email_data.shape[0]

27756

In [93]:
grouped = pd.DataFrame()
grouped['email'] = grouped_email_data.index.values
grouped['opened_emails'] = grouped_email_data.values
grouped.head()

,email,opened_emails
0,01bertoferreira@gmail.com,52
1,07019312.das@gmail.com,1
2,08rosefreire@gmail.com,1
3,09tsuguio@gmail.com,4
4,0marciomodesto@gmail.com,4


In [94]:
final_result = pd.merge(result, grouped, how='left', on='email')
final_result.shape[0]

31991

In [95]:
final_result['opened_emails'] = final_result['opened_emails'].fillna(0).astype(int)

In [96]:
final_result.head().transpose()

,0,1,2,3,4
email,zagg@zagg.com.br,matheus.sfreire@gmail.com,raulg.sales@hotmail.com,prof.sergio.costa@gmail.com,sergioaguiar@yahoo.com
is_paying_student,0,0,0,0,0
age_in_days,407,406,401,403,375
webinar_enrollments,0,0,0,0,0
course_enrollments,1,3,4,68,3
is_checkout,0,2,0,0,2
is_referrer_linkedin,0,0,0,0,0
is_ai,0,0,0,0,0
is_fcop_ud,0,24,20,27,5
is_fcop_cs,0,1,10,6,0


## Reordering features and saving to a file
Final steps are 1) reordering features, leaving the target **`is_paying_student`** as last feature, and 2) saving into a CSV file:

In [97]:
for index, value in enumerate(final_result.columns.values):
    print(index, value)

0 email
1 is_paying_student
2 age_in_days
3 webinar_enrollments
4 course_enrollments
5 is_checkout
6 is_referrer_linkedin
7 is_ai
8 is_fcop_ud
9 is_fcop_cs
10 is_referrer_github
11 is_referrer_facebook
12 is_legal
13 is_mobile
14 is_hire_talent
15 count_visits
16 is_referrer_tecmundo
17 is_robotics
18 is_drive
19 is_referrer_live
20 is_referrer_instagram
21 is_event
22 is_referrer_computerworld
23 is_ndop
24 is_catalog_all
25 is_contact
26 is_referrer_catracalivre
27 is_business
28 is_catalog_nanodegrees
29 is_50back
30 is_payment
31 is_referrer_bing
32 is_home
33 is_nanodegree_home
34 is_success
35 is_referrer_infomoney
36 is_signin
37 is_referrer_anhanguera
38 is_android
39 is_referrer_cbsi
40 is_us
41 is_referrer_google
42 is_fcop_st
43 is_referrer_android
44 is_tech_requirements
45 is_jobs
46 opened_emails


In [98]:
cols = list(final_result)
cols[46], cols[1] = cols[1], cols[46]
final_data = final_result.ix[:,cols]

In [101]:
for index, value in enumerate(final_data.columns.values):
    print(index, value)

0 email
1 opened_emails
2 age_in_days
3 webinar_enrollments
4 course_enrollments
5 is_checkout
6 is_referrer_linkedin
7 is_ai
8 is_fcop_ud
9 is_fcop_cs
10 is_referrer_github
11 is_referrer_facebook
12 is_legal
13 is_mobile
14 is_hire_talent
15 count_visits
16 is_referrer_tecmundo
17 is_robotics
18 is_drive
19 is_referrer_live
20 is_referrer_instagram
21 is_event
22 is_referrer_computerworld
23 is_ndop
24 is_catalog_all
25 is_contact
26 is_referrer_catracalivre
27 is_business
28 is_catalog_nanodegrees
29 is_50back
30 is_payment
31 is_referrer_bing
32 is_home
33 is_nanodegree_home
34 is_success
35 is_referrer_infomoney
36 is_signin
37 is_referrer_anhanguera
38 is_android
39 is_referrer_cbsi
40 is_us
41 is_referrer_google
42 is_fcop_st
43 is_referrer_android
44 is_tech_requirements
45 is_jobs
46 is_paying_student


In [100]:
final_data.to_csv('data_prepared.csv')